<a href="https://colab.research.google.com/github/Jake-Jasper/AI-Capstone-Project/blob/main/Making_local_docs_with_sent_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
    !pip install -Uq sentence-transformers pdfminer.six spacy PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 36.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade --quiet langchain-text-splitters tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.1 MB/s eta 0:00:00


In [ ]:
from pdfminer.high_level import extract_text
from sentence_transformers import util
import numpy as np
from spacy.lang.en import English
import spacy, os,sqlite3, pdfminer, fitz
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import CharacterTextSplitter, TokenTextSplitter
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')

In [ ]:
DB = '/content/drive/MyDrive/12-4-24-mpnet-token-100.db'

In [ ]:
# Establish a connection to the SQLite database
conn = sqlite3.connect(DB)

# Create a cursor object
cursor = conn.cursor()

# Execute SQL queries, change embedding type to somethiiing more suitbale in future
# In final version normalise this by having a documents table, not need to have title here
cursor.execute('''CREATE TABLE IF NOT EXISTS documents (
                    id INTEGER PRIMARY KEY,
                    document_ID INTEGER,
                    document_title TEXT,
                    sentence TEXT,
                    embedding BLOB)''')

# Commit the transaction
conn.commit()
conn.close()


In [ ]:
def read_pdf(file):
    text = extract_text(file)
    return text

In [ ]:
DIR = "/content/drive/MyDrive/WRAP food reports"

In [ ]:
#mixedbread-ai/mxbai-embed-large-v1 ; sentence-transformers/all-MiniLM-L6-v2; 'sentence-transformers/all-mpnet-base-v2'
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
MAX_TOKEN = 100
overlap = round(MAX_TOKEN * 0.1)
# Establish a connection to the SQLite database
conn = sqlite3.connect(DB)

# Create a cursor object
cursor = conn.cursor()

for num, title in enumerate(sorted(os.listdir(DIR))):

    path = DIR + "/" + title
    # read doc
    text = read_pdf(path)
    text = text.replace("\n", "")
    # split into sentences
    #doc = nlp(text)
    text_splitter = TokenTextSplitter(chunk_size=MAX_TOKEN, chunk_overlap=20)
    texts = text_splitter.split_text(text)
    # loop to embed sentences
    for sent in tqdm(texts):
        embedding = model.encode(sent)
        #num+1 so it is not indexed from 0
        cursor.execute("""INSERT INTO documents (document_id,
                                                document_title,
                                                sentence,
                                                embedding) VALUES (?, ?, ?, ?)""",
                                                 (num+1, title, sent, embedding))

conn.commit()
conn.close()

100%|██████████| 330/330 [02:49<00:00,  1.94it/s]


In [ ]:

def find_top_k_relevance(question=["No question"], model=None, n=10, db=DB):
  q = model.encode(question)
  scores = {}
  conn = sqlite3.connect(db)

# Create a cursor object
  cursor = conn.cursor()
  # load row by row
  cursor.execute('SELECT * FROM documents')
  for row in cursor:
    scores[row[0]] = util.pytorch_cos_sim(q, np.frombuffer(row[4], dtype=np.float32)).numpy()

  return dict(sorted(scores.items(), key = lambda x: x[1], reverse = True)[:n])


find_top_k_relevance(question =["How should I store Bananas?"], model=model)

{3484: array([[0.7250447]], dtype=float32),
 3482: array([[0.70542157]], dtype=float32),
 3480: array([[0.650809]], dtype=float32),
 3485: array([[0.6437081]], dtype=float32),
 3481: array([[0.61441773]], dtype=float32),
 3479: array([[0.60196793]], dtype=float32),
 3492: array([[0.601953]], dtype=float32),
 3483: array([[0.5850687]], dtype=float32),
 3430: array([[0.5828389]], dtype=float32),
 3493: array([[0.5783164]], dtype=float32)}

In [ ]:
import pprint
conn = sqlite3.connect(DB)
cursor = conn.cursor()
cursor.execute("""SELECT * FROM documents
                 WHERE id = 3431""")
rows = cursor.fetchone()
conn.close()
pprint.pprint(f"{rows[3]} context: {rows[2]}")

(' bananas (WRAP, 2011).  Morrison’s sells packaged and free-flow bananas, '
 'presenting single loose fruits in a box so customers can buy single/small '
 'quantities.  Tesco and Morrisons offer free fruit for kids whilst their '
 'parent/guardian shops in-store, which could be an effective way of using '
 'single loose bananas. Refrigerating broccoli at 4°C maintained shelf-life '
 'for 10 days, regardless of whether the heads were stored in Modified '
 'Atmosphere Packaging ( context: 14 - WRAP-Evidence Review Plastic Packaging '
 'and Fresh Produce 171218.pdf')




## Question list (30) - out of date, use list in eval notebook.
Have the relevant document for each so you can measure, recall, precision etc

Best to complete this after asking the question to the system?

|Doc_ID|Name|Relevant_Q|
|---|---|---|
|1|Evidence & insights - reducing household food waste & plastic packaging (packaged vs loose)|1,2,3,4,5,6,7,9|
|2|Food waste collection guidance |8, 17|
|3|Food Waste Reduction Roadmap - Executive Summary|9, 16|
|4|Food Waste Reduction Roadmap - Hospitality|9|
|5|Food Waste Reduction Roadmap - Manufacturers|9|
|6|Food Waste Reduction Roadmap - Primary Producers|9,10|
|7|Food Waste Reduction Roadmap - Retail|9,11|
|8|Identifying impacts from food and farm digestates|12,13|
|9|Industry Guidance - Dealing with Household Food Waste at AD Facilities - Management of Liners|14,8, 15|
|10|Literature review - relationship between household food waste collection and food waste prevention|8, 16, 17, 18, 19|
|11|The food waste reduction roadmap progress report 2022|9|
|12|Towards the 2030 Food Waste Commitment -- setting our coalition baseline|14|
|13|Upscaling farm food waste measurement in the UK|22|
|14|WRAP-Evidence Review Plastic Packaging and Fresh Produce 171218|3, 4, 14|

1. "How should I store bananas?"
2. "What temperature should I keep milk?"
3. "How do I increase shelf life of lettuce?
4. "How does selling fresh produce loose, affect waste?"
5. "What is a best-before date for?"
6. "What is a use-by date for?"
7. "How does removing date labels help?"
8. "What are the reasons to not participate in a food waste scheme?"
9. "What is the UN's sustainable development goal with respect to food?"
10. "Which document pertains to primary producers and food waste?"
11. "Which document pertains to retailers and food waste?"
12. "What is the most common use for digestate?"
13. "What affects the costs of valorisation?"
14. "How many tonnes of food waste are there each year?"
15. "What is the benefit of caddy liners?"
16. "How to prevent food loss?"
17. "How to prevent food waste?"
18. "How does composting affect food waste?"
19. "What are the main drivers of food waste? got to here!"
20. "Online shopping and food waste"
21. "What is involved with Courthold 2030?"
22. "How do you measure in field food waste?"
